In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import h5py
from math import pi
from scipy.stats import zscore

from analysis.config import birds, h5_path_dict, pickle_dir
from analysis.ExpData import ExpData
from analysis.utils import get_fr

# Collect population patterns and save into pickle file
Visit patterns are (visits, neurs, frames) size

### Parameters

In [2]:
window = int(20*2)
fr_smoothing_window = int(20*0.25)
l2_normalization = True
pfile_name = "population_patterns.p"

### Data formatting

In [4]:
data = {}
for key in h5_path_dict.keys():
    for fil in range(len(h5_path_dict[key])):
        fil_string = h5_path_dict[key][fil].as_posix()
        print(fil_string)
        f = h5py.File(fil_string, 'r')
        exp_data = ExpData(f)
        data[fil_string] = {}
        c_visits, r_visits, noncr_visits = exp_data.get_cr_visits()
        hopcentered_visits = exp_data.get_hopcentered_visits(window)
        fr = get_fr(exp_data.spikes, fr_smoothing_window)
        if l2_normalization:
            scaling = np.square(np.linalg.norm(fr, axis=1))
            #scaling = np.linalg.norm(fr, axis=1)
            fr /= scaling[:, None]
        visit_patterns = np.zeros(
            (hopcentered_visits.shape[0], exp_data.num_neurs, window*2 + 1)
            )
        for i, visit in enumerate(hopcentered_visits):
            visit_pattern = fr[:, visit]
            visit_pattern[:, visit == -1] = np.nan
            visit_patterns[i] = visit_pattern            
        data[fil_string]['c_visits'] = c_visits
        data[fil_string]['r_visits'] = r_visits
        data[fil_string]['noncr_visits'] = noncr_visits
        data[fil_string]['fr'] = fr
        data[fil_string]['visit_patterns'] = visit_patterns
        data[fil_string]['visit_times'] = hopcentered_visits
with open(pickle_dir / pfile_name, 'wb') as f: 
    pickle.dump(data, f)

/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184331_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned185033_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184713_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184430_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned144233_09112019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184526_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184946_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20200930/2020-09-30-11-00-47_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithDLCAndAnnotations164614_10222020.mat


/home/chingf/anaconda3/envs/analysis-env/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20201009/2020-10-09-11-40-59_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithDLCAndAnnotations140712_10202020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20201012/2020-10-12-11-43-36_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithDLCAndAnnotations161757_10232020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20200918/2020-09-18-11-50-00_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithDLCAndAnnotations153159_10222020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20200908/2020-09-08-12-02-54_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithDLCAndAnnotations154721_10142020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20200910/2020-09-10-12-00-48_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithDLCAndAnnotations161306_10142020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20201007/2020-10-07-11-48-53_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWit